In [1]:
import numpy as np
import pandas as pd
import os
import sys
import warnings
warnings.filterwarnings("ignore")
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
cwd = os.getcwd()

counter = 0
print(counter, cwd)
while not cwd.endswith("drone_steering") and counter < 10:
    os.chdir('..')
    cwd = os.getcwd()
    counter = counter + 1
    print(counter, cwd)

0 C:\Users\p.schambach\Desktop\DSR\drone_steering\models\playground
1 C:\Users\p.schambach\Desktop\DSR\drone_steering\models
2 C:\Users\p.schambach\Desktop\DSR\drone_steering


In [2]:
from app_local.module import DataEnsembler, GestureTransformer

de = DataEnsembler(120)
de.investigate_available_datafiles(data_dir='data/gesture/',is_frame_based=True)
de.load_data()
de.assemble_data(tolerance_range=2)
de.display_information()

i: 0 	shape X: (603, 18, 16) 	shape y: (603,) 	count: 101
i: 1 	shape X: (519, 18, 16) 	shape y: (519,) 	count: 89
i: 2 	shape X: (498, 18, 16) 	shape y: (498,) 	count: 119
i: 3 	shape X: (650, 18, 16) 	shape y: (650,) 	count: 230
i: 4 	shape X: (508, 18, 16) 	shape y: (508,) 	count: 147
i: 5 	shape X: (466, 18, 16) 	shape y: (466,) 	count: 146
i: 6 	shape X: (601, 18, 16) 	shape y: (601,) 	count: 159
i: 7 	shape X: (440, 18, 16) 	shape y: (440,) 	count: 127
i: 8 	shape X: (490, 18, 16) 	shape y: (490,) 	count: 110
i: 9 	shape X: (476, 18, 16) 	shape y: (476,) 	count: 118
i: 10 	shape X: (492, 18, 16) 	shape y: (492,) 	count: 121
i: 11 	shape X: (436, 18, 16) 	shape y: (436,) 	count: 87
i: 12 	shape X: (466, 18, 16) 	shape y: (466,) 	count: 126
i: 13 	shape X: (551, 18, 16) 	shape y: (551,) 	count: 127
i: 14 	shape X: (550, 18, 16) 	shape y: (550,) 	count: 114
i: 15 	shape X: (549, 18, 16) 	shape y: (549,) 	count: 115
i: 16 	shape X: (541, 18, 16) 	shape y: (541,) 	count: 92
i: 17 	sha

In [3]:
#from app_local.module import DataResampler

In [58]:
class DataResampler():
    
    def __init__(self):
        pass
        
    
    def fit(self, X, y):
        if X.shape[0] != y.shape[0]:
            raise AssertionError("First Dimension of Argument X and y have to be equal")
            
        self.orig_y = y.copy().astype("int32")
        self.orig_X = X.copy()
        
        self.X = self.orig_X.copy()
        self.y = self.orig_y.copy()
            
        self.labels = sorted(set(y.astype("int32")))
        
        maxl = max(self.labels)
        
        self.indices = {}
        for i in self.labels:
            self.indices[i] = np.where(np.isclose(y,i))[0]
            
        n = self.orig_X.shape[0]
        v = np.zeros(n)
        s = np.zeros(n)

        for i in range(n):
            v[i] = np.abs(self.orig_X[i,:,:].var(axis = 0)).sum()
            s[i] = np.abs(np.diff(self.orig_X[i,:,:], axis = 0)).sum()
            
        self.variances = v
        self.abs_diffs = s
            
            
    def restrict_zero_labels(self, n, criterion = None):
        y = self.orig_y.copy()
        
        n0 = len(self.indices[0])
        print("-------------------------------------")
        print("Current number of zero-labels:", n0)
        
        if n0 <= n:
            print("Target number of zero-labels is bigger than current number. No changes!")
        elif n < 0:
            print("Target number of zero-labels was negative. Treated as zero!")
        else:
            if criterion == 'abs_diff_lowest':
                zero_idx = self.indices[0][np.argsort(self.abs_diffs[self.indices[0]])[0:n]]
            elif criterion == 'abs_diff_highest':
                zero_idx = self.indices[0][np.argsort(self.abs_diffs[self.indices[0]])[-n:]]
            elif criterion == 'variances_lowest':
                zero_idx = self.indices[0][np.argsort(self.variances[self.indices[0]])[0:n]]
            elif criterion == 'variances_highest':
                zero_idx = self.indices[0][np.argsort(self.variances[self.indices[0]])[-n:]]
            else:
                zero_idx = np.random.choice(self.indices[0], n, replace=False)
            
            other_labels_idx = np.concatenate([self.indices[i] for i in self.indices.keys() if i != 0])
            new_idx = sorted(np.concatenate([zero_idx, other_labels_idx]))
            
        
        print("New number of zero-labels{0}:".format("" if criterion is None else " (selected with criterion=\"{0}\")".format(criterion)), n)
        
        self.X = self.orig_X[new_idx,:,:]
        self.y = self.orig_y[new_idx]

        return self.X, self.y 
    
    
    def upsample(self, n = None, exceptions = None):
        if n is None:
            n = len(self.indices[0])
        elif n < 0:
            n = 0
        
        a = {} 
        b = {}
        idx = {}

        if exceptions is None:
            exceptions = []
            
        for i in self.indices.keys():
            if i in exceptions:
                a[i] = len(self.indices[i])
                b[i] = 0
            else:
                a[i] = min(n, len(self.indices[i]))
                diff = n - len(self.indices[i])
                b[i] = max(0, diff)
            
            idx_a = np.random.choice(self.indices[i], a[i], replace = False)
            idx_b = np.random.choice(self.indices[i], b[i], replace = True)
            
            idx[i] = sorted(np.concatenate([idx_a, idx_b]))
            
        
        upsampled_idx = sorted(np.concatenate([idx[i] for i in idx]))
        
        self.X = self.orig_X[upsampled_idx,:,:]
        self.y = self.orig_y[upsampled_idx]
        
        return self.X, self.y  
    
    def display_information(self):
        df = pd.DataFrame(columns=["abs_orig","pct_orig","abs_current","pct_current"],index = self.labels)
        
        n = len(self.y)
        
        for l in self.labels:
            nl = len(self.y[self.y == l])
            
            df.loc[l,"abs_orig"] = len(self.indices[l])
            df.loc[l,"pct_orig"] = np.round(100*len(self.indices[l])/len(self.orig_y),2)
            df.loc[l,"abs_current"] = nl
            df.loc[l,"pct_current"] = np.round(100*nl/n,2)
              
        return df
    
    
    def train_test_split(self, fraction):
        
        remove_idx = {}
        keep_idx = {}
        for l in self.labels:
            nl = len(self.indices[l])
            rn = int(np.round(nl*fraction))
            remove_idx[l] = sorted(np.random.choice(self.indices[l], rn, replace = False))
            keep_idx[l] = sorted(set(self.indices[l]) - set(remove_idx[l]))
          
        remove = sorted(np.concatenate([remove_idx[i] for i in remove_idx]))
        keep = sorted(np.concatenate([keep_idx[i] for i in keep_idx]))
        
        remove = np.random.permutation(remove)
        keep = np.random.permutation(keep)
        
        self.X = self.orig_X[keep,:,:]
        self.y = self.orig_y[keep]
        
        x_test = self.orig_X[remove,:,:]
        y_test = self.orig_y[remove]
        
        return x_test, y_test
    
    
    
    
    

In [18]:
a = np.random.choice(np.arange(100), 50, replace = False)
a
i = np.argsort(a)
a[i]

array([ 0,  2,  3,  8, 10, 11, 12, 14, 16, 18, 19, 20, 25, 27, 32, 33, 35,
       36, 37, 38, 39, 40, 43, 44, 45, 46, 47, 48, 50, 54, 55, 59, 60, 62,
       64, 65, 66, 67, 71, 73, 75, 82, 83, 84, 87, 91, 92, 95, 97, 99])

In [59]:
dr = DataResampler()

In [60]:
dr.fit(de.X, de.y)
dr.display_information()

,abs_orig,pct_orig,abs_current,pct_current
0,13894,76.39,13894,76.39
1,701,3.85,701,3.85
2,576,3.17,576,3.17
3,991,5.45,991,5.45
4,677,3.72,677,3.72
5,660,3.63,660,3.63
6,689,3.79,689,3.79


In [61]:
crits = ["abs_diff_lowest","abs_diff_highest","variances_lowest","variances_highest"]

for crit in crits:
    dr.restrict_zero_labels(n=2000,criterion=crit)
    print(dr.display_information())

-------------------------------------
Current number of zero-labels: 13894
New number of zero-labels (selected with criterion="abs_diff_lowest"): 2000
  abs_orig pct_orig abs_current pct_current
0    13894    76.39        2000       31.78
1      701     3.85         701       11.14
2      576     3.17         576        9.15
3      991     5.45         991       15.75
4      677     3.72         677       10.76
5      660     3.63         660       10.49
6      689     3.79         689       10.95
-------------------------------------
Current number of zero-labels: 13894
New number of zero-labels (selected with criterion="abs_diff_highest"): 2000
  abs_orig pct_orig abs_current pct_current
0    13894    76.39        2000       31.78
1      701     3.85         701       11.14
2      576     3.17         576        9.15
3      991     5.45         991       15.75
4      677     3.72         677       10.76
5      660     3.63         660       10.49
6      689     3.79         689      

In [7]:
a,b = dr.train_test_split(0.2)

In [8]:
print(a.shape, b.shape)
dr.display_information()

(3637, 18, 16) (3637,)


,abs_orig,pct_orig,abs_current,pct_current
0,13894,76.39,11115,76.39
1,701,3.85,561,3.86
2,576,3.17,461,3.17
3,991,5.45,793,5.45
4,677,3.72,542,3.72
5,660,3.63,528,3.63
6,689,3.79,551,3.79


In [9]:
dr.restrict_zero_labels(500)
dr.display_information()

-------------------------------------
Current number of zero-labels: 13894
New number of zero-labels: 500


,abs_orig,pct_orig,abs_current,pct_current
0,13894,76.39,500,10.43
1,701,3.85,701,14.62
2,576,3.17,576,12.02
3,991,5.45,991,20.67
4,677,3.72,677,14.12
5,660,3.63,660,13.77
6,689,3.79,689,14.37


In [10]:
dr.upsample()
dr.display_information()

,abs_orig,pct_orig,abs_current,pct_current
0,13894,76.39,13894,14.29
1,701,3.85,13894,14.29
2,576,3.17,13894,14.29
3,991,5.45,13894,14.29
4,677,3.72,13894,14.29
5,660,3.63,13894,14.29
6,689,3.79,13894,14.29


In [11]:
print(dr.X.shape, dr.y.shape, dr.orig_X.shape, dr.orig_y.shape)

(97258, 18, 16) (97258,) (18188, 18, 16) (18188,)


In [12]:
dr.upsample(200)
dr.display_information()

,abs_orig,pct_orig,abs_current,pct_current
0,13894,76.39,200,14.29
1,701,3.85,200,14.29
2,576,3.17,200,14.29
3,991,5.45,200,14.29
4,677,3.72,200,14.29
5,660,3.63,200,14.29
6,689,3.79,200,14.29


In [13]:
dr.upsample(10000, exceptions=[0])
dr.display_information()

,abs_orig,pct_orig,abs_current,pct_current
0,13894,76.39,13894,18.8
1,701,3.85,10000,13.53
2,576,3.17,10000,13.53
3,991,5.45,10000,13.53
4,677,3.72,10000,13.53
5,660,3.63,10000,13.53
6,689,3.79,10000,13.53
